# Analyse de cooccurrences et de collocations
NLTK offre des fonctions pour détecter les cooccurrences/collocations d'un texte.

Nous avons une cooccurrence lorsque la présence d’un mot dans un texte donne une indication de la présence d’un autre mot. Une collocation est une forme plus restreinte de cooccurrence. Habituellement, une collocation est constituée de mots consécutifs dont la signification ne peut pas être dérivée de la signification des mots individuels.

Dans le cadre de ce cours, on ne fait pas de distinction entre les deux concepts et on s'intéresse de manière générale aux mots corrélés entre eux.

Nous allons illustrer les différentes techniques présentées dans la capsule vidéo sur le site du cours pour détecter des cooccurrences à partir du corpus de Brown.

## 1. Analyse de cooccurrences par la fréquence
### Bigrammes

La fréquence de N-grammes est un critère qui peut utiliser pour identifier les cooccurrences. Cependant, il est important de faire un bon filtrage pour éliminer le bruit. Car une fréquence élevée n'est pas toujours un bon indicateur. On compare ici les N-grammes les plus fréquents avec ceux obtenus en filtrant à partir des étiquettes grammaticales (POS tags).

In [7]:
import nltk
import pandas as pd
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

nb_results = 25
brown_words = nltk.corpus.brown.words()
brown_tagged_words = nltk.corpus.brown.tagged_words('ca01', tagset='universal')

freq_results = dict()

bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(brown_words)
freq_results["freq"] = finder.nbest(bigram_measures.raw_freq, nb_results)

tag_patterns = [("NOUN", "NOUN"), ("ADJ", "NOUN"), ("VERB", "NOUN")]
finder = BigramCollocationFinder.from_words(brown_tagged_words)
raw_coocs = finder.nbest(bigram_measures.raw_freq, 1000)
filtered_coocs = list()
for (w1, tag1), (w2, tag2) in raw_coocs:
    if (tag1, tag2) in tag_patterns:
        filtered_coocs.append((w1, w2))
freq_results["freq-pos"] = filtered_coocs[:nb_results]

display(pd.DataFrame(freq_results))

,freq,freq-pos
0,"(of, the)","(Fulton, County)"
1,"(,, and)","(Highway, Department)"
2,"(., The)","(school, superintendent)"
3,"(in, the)","(Allen, Jr.)"
4,"(,, the)","(Blue, Ridge)"
5,"(., ``)","(Department, source)"
6,"(to, the)","(Executive, Committee)"
7,"('', .)","(Fulton, legislators)"
8,"(;, ;)","(Ivan, Allen)"
9,"(., He)","(Miller, County)"


**Trigrammes** - On peut faire la même analyse avec les différents N-grammes de mots.



In [8]:
from nltk.collocations import TrigramAssocMeasures, TrigramCollocationFinder

nb_results = 25
results = dict()

trigram_measures = TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(brown_words)
results["freq"] = finder.nbest(trigram_measures.raw_freq, nb_results)

# Quelques patrons de POS tags possibles
tag_patterns = [("NOUN", "NOUN", "NOUN"), ("ADJ", "NOUN", "NOUN"), ("ADJ", "ADJ", "NOUN"), 
                ("NOUN", "ADP", "NOUN"), ("VERB", "NOUN", "NOUN"), ("VERB", "ADP", "NOUN")]
finder = TrigramCollocationFinder.from_words(brown_tagged_words)
raw_coocs = finder.nbest(trigram_measures.raw_freq, 1000)
filtered_coocs = list()
for (w1, tag1), (w2, tag2), (w3, tag3) in raw_coocs:
    if (tag1, tag2, tag3) in tag_patterns:
        filtered_coocs.append((w1, w2, w3))
results["freq-pos"] = filtered_coocs[:nb_results]

display(pd.DataFrame(results))

,freq,freq-pos
0,"('', ?, ?)","(Highway, Department, source)"
1,"('', ., ``)","(Ivan, Allen, Jr.)"
2,"(,, and, the)","(Rural, Roads, Authority)"
3,"(., It, is)","(State, Highway, Department)"
4,"(., It, was)","(bit, of, trouble)"
5,"(., In, the)","(new, school, superintendent)"
6,"(?, ?, ``)","(rural, roads, bonds)"
7,"(., ``, I)","($10, per, day)"
8,"('', !, !)","(Alpharetta, prison, farms)"
9,"('', ,, he)","(Ask, jail, deputies)"


## 2. Analyse de cooccurrences avec l'information mutuelle
On présente ici les résultats obtenus en retenant les bigrammes dont la valeur d'information mutuelle (PMI - Pointwise mutual information) est la plus élevée.

On sait que la mesure d'information mutuelle est sensible aux faibles fréquences. On présente également dans le tableau les résultats lorsqu'on filtre selon le nombre d'occurrences d'un bigramme.

In [9]:
nb_results = 25

bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(brown_words)

pmi_results = dict()
pmi_results["pmi"] = finder.nbest(bigram_measures.pmi, nb_results)

finder.apply_freq_filter(2)
pmi_results["pmi-filtre-2"] = finder.nbest(bigram_measures.pmi, nb_results)

finder.apply_freq_filter(5)
pmi_results["pmi-filtre-5"] = finder.nbest(bigram_measures.pmi, nb_results)

finder.apply_freq_filter(10)
pmi_results["pmi-filtre-10"] = finder.nbest(bigram_measures.pmi, nb_results)

display(pd.DataFrame(pmi_results))

,pmi,pmi-filtre-2,pmi-filtre-5,pmi-filtre-10
0,"($10,000-per-year, French-born)","(Ablard, Corne)","(Baton, Rouge)","(Hong, Kong)"
1,"($79.89, nothing-down)","(Agatha, Christie)","(Dolce, Vita)","(Viet, Nam)"
2,"($8.50, tab)","(Averell, Harriman)","(Neutral, Tones)","(Pathet, Lao)"
3,"('low, nigras)","(Ballistic, Missile)","(Notre, Dame)","(Simms, Purdew)"
4,"(0.5-mv./m., 50-percent)","(Bon, jour)","(Sante, Fe)","(El, Paso)"
5,"(0.78, mEq)","(Bryn, Mawr)","(deja, vue)","(Lo, Shu)"
6,"(1,100, circumscriptions)","(Champs, Elysees)","(boa, constrictor)","(Internal, Revenue)"
7,"(1,257,700, non-farm)","(Citizens', Councils)","(Gratt, Shafer)","(Puerto, Rico)"
8,"(11-inch, headroom)","(Computing, Allotments)","(Pulley, Bey)","(Saxon, Shore)"
9,"(11-shot, hammerless)","(Cooch, Terpers)","(Walnut, Trees)","(Export-Import, Bank)"


## 3. Analyse de cooccurrences avec d'autres mesures statistiques
Le chapitre 5 du livre de Chris Manning présente d'autres mesures qui peuvent être utilisées pour détecter/filtrer les cooccurences. NLTK offre la plupart de ces mesures. Le tableau suivant présente un échantillon des résultats obtenus avec ces mesures. Comme on peut le voir, la qualité des résultats est très variable.

In [10]:
nb_results = 20

bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(brown_words)

other_results = dict()
finder.apply_freq_filter(5)
other_results["student"] = finder.nbest(bigram_measures.student_t, nb_results)
other_results["likelihood ratio"] = finder.nbest(bigram_measures.likelihood_ratio, nb_results)
other_results["chi sq"] = finder.nbest(bigram_measures.chi_sq, nb_results)

display(pd.DataFrame(other_results))

,student,likelihood ratio,chi sq
0,"(of, the)","(., The)","(Baton, Rouge)"
1,"(., The)","(;, ;)","(Dolce, Vita)"
2,"(,, and)","(?, ?)","(Hong, Kong)"
3,"(in, the)","(of, the)","(Lo, Shu)"
4,"(., ``)","(., He)","(Notre, Dame)"
5,"(;, ;)","(., ``)","(deja, vue)"
6,"('', .)","(in, the)","(Los, Angeles)"
7,"(., He)","(,, and)","(Puerto, Rico)"
8,"(?, ?)","(., It)","(Neutral, Tones)"
9,"(., It)","('', .)","(Viet, Nam)"


On peut toutefois améliorer un peu les résultats en filtrant les mots outils (stop words) et les caractères individuels.



In [14]:
ignored_words = nltk.corpus.stopwords.words('english')
finder.apply_word_filter(lambda w: len(w) < 2 or w.lower() in ignored_words)

other_results = dict()
other_results["student"] = finder.nbest(bigram_measures.student_t, nb_results)
other_results["likelihood ratio"] = finder.nbest(bigram_measures.likelihood_ratio, nb_results)
other_results["chi sq"] = finder.nbest(bigram_measures.chi_sq, nb_results)

display(pd.DataFrame(other_results))


,student,likelihood ratio,chi sq
0,"(United, States)","(United, States)","(Baton, Rouge)"
1,"(New, York)","(New, York)","(Dolce, Vita)"
2,"(per, cent)","(per, cent)","(Hong, Kong)"
3,"(years, ago)","(Rhode, Island)","(Lo, Shu)"
4,"('', --)","(years, ago)","(Notre, Dame)"
5,"(Rhode, Island)","(U., S.)","(deja, vue)"
6,"(could, see)","(Los, Angeles)","(Los, Angeles)"
7,"(``, I'm)","(White, House)","(Puerto, Rico)"
8,"(``, Well)","(Peace, Corps)","(Neutral, Tones)"
9,"(``, Oh)","(World, War)","(Viet, Nam)"


## 4. Cooccurrences à l'intérieur d'une fenêtre de mots
On termine en rappelant que les cooccurrences n'ont pas à être des mots consécutifs. Ils peuvent être dans une même phrase ou à l'intérieur d'une fenêtre de mots. On présente ici quelques résultats pour illustrer les relations entre ces mots.

In [15]:
def cooc_within_window(words, window_size=20, nb_results=25, min_freq=5):
    finder = BigramCollocationFinder.from_words(words, window_size=window_size)
    finder.apply_freq_filter(min_freq)
    ignored_words = nltk.corpus.stopwords.words('english')
    finder.apply_word_filter(lambda w: w.lower() in ignored_words)
    return finder.nbest(bigram_measures.pmi, nb_results)

window_results = dict()
window_results["Fenetre-5"] = cooc_within_window(brown_words, window_size=5)
window_results["Fenetre-10"] = cooc_within_window(brown_words, window_size=10)
window_results["Fenetre-20"] = cooc_within_window(brown_words, window_size=20)

display(pd.DataFrame(window_results))

,Fenetre-5,Fenetre-10,Fenetre-20
0,"(Baton, Rouge)","(Consulting, Engineer)","(Farmer, dell)"
1,"(Dolce, Vita)","(Hwang, Pah)","(po'k, skippers)"
2,"(Lauro, Bosis)","(vs, vs)","(Consulting, Engineer)"
3,"(Mollie, Mutton)","(Presiding, Elder)","(Consulting, Senior)"
4,"(Neutral, Tones)","(Baton, Rouge)","(Dealing, faro)"
5,"(Notre, Dame)","(Dolce, Vita)","(conformational, entropy)"
6,"(Sante, Fe)","(Mollie, Mutton)","(Engineer, Senior)"
7,"(deja, vue)","(Lauro, Bosis)","(Engineer, Engineer)"
8,"(boa, constrictor)","(Notre, Dame)","(vs, vs)"
9,"(Gratt, Shafer)","(Neutral, Tones)","(Hoot, Mon-Goddess)"
